REPLICATION OF THIRD CHAPTER SIMULATIONS

In [463]:
import os
import numpy as np
from numpy import random as rnd
import pandas as pd
import math
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import scipy
from scipy.stats.distributions import chi2
from scipy.optimize import minimize
import statistics
from statistics import mean

In [464]:
# Calculate the minimally impermissible rate of localisation  
def mirl(alpha, beta):
    if alpha + 1/2 > beta:
        deltabar = ((2 * beta) - 1 ) / (2 * (alpha + beta) + 1)
    else:
        deltabar = alpha / ((2 * alpha) + 1)
    return deltabar

# Joint density of X and Z
def joint_density(J, x, z, alpha, n, delta, eig_const):
    fxz = 1
    for j in range(2, J+1):
        fxz += (np.sqrt(eig_const)/((j-1) ** (alpha/2))) * (2 * np.cos((j-1) * np.pi * x) * np.cos((j-1) * np.pi * z)) / (n ** (delta/2))
    return fxz

# Structural function
def phi_x(J, x, beta):
    phi = 0.5
    for j in range(2, J+1):
        phi +=  np.sqrt(2) * (((j-1) * np.pi) ** (-beta/2)) * np.cos((j-1) * np.pi * x)
    return phi

# Eigenvalues of T
def eigenvalues(J, alpha, n, delta, eig_const):
    eigs = np.zeros((J, 1))
    eigs[0] = 1
    for j in range(2, J+1):
        eigs[j-1] =  eig_const / (((j-1) ** alpha) * (n ** delta))
    return eigs

# Fourier coefficients of the structural function
def fourier_coeffs(J, z, y, eigs, n):
    rjvec = np.zeros((J, 1))
    rjvec[0] = np.mean(y)
    for j in range(2, J+1):
        rjvec[j-1] =  np.sqrt(2) * np.sqrt(eigs[j-1]) * np.dot(y.T, np.cos((j-1) * np.pi * z)) / n
    return rjvec

# Calculate integrated mean squared error for given estimator
def integrated_mean_squared_error(estimator_values, true_function):
    estimator_stacked = np.column_stack(estimator_values)
    integrated_squared_bias = np.round(np.mean(( np.mean(estimator_stacked, axis = 1) - true_function) ** 2), 4)
    integrated_variance = np.round(np.mean(np.var(estimator_stacked, axis = 1)), 4)
    integrated_mean_squared_error = np.round(integrated_squared_bias +  integrated_variance, 4)
    return integrated_squared_bias, integrated_variance, integrated_mean_squared_error

In [ ]:
import numpy as np
np.random.seed(1029384)

# SIMULATION PARAMETERS

num_sims = 500
n_grid = [200, 500]


alpha_grid = [2, 4]
beta_grid = [2, 4]
c_grid = [0, 0.5, 1 ,2]

eig_const = 0.2

# Grids for caluclation of integrated mean squared errors
xgrid = np.arange(0.02, 1.0, 0.02)
nxgrid = len(xgrid)


results_columns = ['n', 'delta', 'alpha', 'beta', 'SC_ISQB', 'SC_IVAR', 'SC_IMSE', 'TK_ISQB', 'TK_IVAR', 'TK_IMSE']
results_df = pd.DataFrame(columns = results_columns)

# Initialise lists for results
values_spectral_cutoff = []
values_tikhonov = []


[(beta, alpha) for beta in beta_grid for alpha in alpha_grid]

for c in c_grid:
    deltabar = mirl(alpha, beta)
    delta = c * deltabar

    for n in n_grid:

        initial_guess = [0, 0]
        result = minimize(fun = lambda x: -joint_density(J, x[0], x[1], alpha, n, delta, eig_const), x0 = initial_guess, method = 'SLSQP')
        maximized_x, maximized_z = result.x
        maximum_value_fxz = -result.fun

        for sim in range(num_sims):

            jj = 1
            x = np.zeros(n)
            z = np.zeros(n)

            # Simulate data
            while jj < n + 1:
                target = np.random.rand(2)
                zval = maximum_value_fxz * np.random.rand()
                val = joint_density(J, target[0], target[1], alpha, n, delta, eig_const)
                istrue = val > zval
                if not istrue:
                    x[jj-1] = target[0]
                    z[jj-1] = target[1]
                else:
                    x[jj-1] = target[0]
                    z[jj-1] = target[1]
                    jj += 1

            # Combine x and z into data
            data = np.column_stack((x, z))
            data = data[data[:, 0].argsort()]

            # Generate endogenous errors
            zregJ = 10
            zreg = np.zeros((n, zregJ))
            for kk in range(1, zregJ + 1):
                zreg[:, kk - 1] = np.sqrt(2) * np.cos((kk - 1) * np.pi * z)

            u = x - zreg @ (np.linalg.inv(zreg.T @ zreg) @ zreg.T @ x) + np.random.normal(0, 0.3, n)

            # Calculate phix
            phix = phi_x(J, x, beta)
            y = phix + u

            # Calculate eigs
            eigs = np.zeros(J)
            eigs[0] = 1
            for j in range(2, J + 1):
                eigs[j - 1] = 0.2 / (((j - 1) ** alpha) * (n ** delta))

            # Calculate rjvec
            rjvec = np.zeros(J)
            rjvec[0] = np.mean(y)
            for jj in range(2, J + 1):
                rjvec[jj-1] = np.sqrt(2) * np.sqrt(eigs[jj-1]) * (y @ np.cos((jj - 1) * np.pi * z) / n)

            # Calculate estcoeffs
            estcoeffs = rjvec / eigs

            # Calculate phihatsumvals
            phihatsumvals = np.zeros((n, J))
            phihatsumvals[:, 0] = np.repeat(estcoeffs[0], n)
            for kk in range(2, J+1):
                phihatsumvals[:, kk-1] = estcoeffs[kk-1] * np.sqrt(2) * np.cos((kk-1) * np.pi * x)

            # Calculate phiJ and phihatscmsevec
            phiJ = 10
            phihatscmsevec = np.zeros(phiJ)
            for kk in range(1, phiJ + 1):
                phihatscsumk = phihatsumvals[:, 0:kk]
                phihatsck = np.sum(phihatscsumk, axis=1)
                errorphihatsc1 = (phix - phihatsck) ** 2
                phihatscmsevec[kk-1] = np.sum(errorphihatsc1) / n

            msescmin, I1sc = np.min(phihatscmsevec), np.argmin(phihatscmsevec)
            phihatscsum = phihatsumvals[:, 0:I1sc]
            phihatsc = np.sum(phihatscsum, axis=1)

            # TIKHONOV ESTIMATOR
            anvec = np.logspace(-3, 0, 200)
            estcoeffstk = np.zeros((len(anvec), J))
            estcoeffstk[:, 0] = np.repeat(rjvec[0], len(anvec)) / (1 + anvec)
            for jj in range(1, J):
                estcoeffstk[:, jj] = np.repeat(rjvec[jj], len(anvec)) / (eigs[jj] + anvec)

            # Calculate basisfunvals
            basisfunvals = np.zeros((n, J))
            basisfunvals[:, 0] = 1
            for jj in range(1, J):
                basisfunvals[:, jj] = np.sqrt(2) * np.cos((jj) * np.pi * x)

            # Calculate phihattkmsevec
            phihattkmsevec = np.zeros(len(anvec))
            for kk in range(0, len(anvec)):
                phihattk = basisfunvals @ estcoeffstk[kk, :].T
                errorphihattk1 = (phix - phihattk) ** 2
                phihattkmsevec[kk] = np.sum(errorphihattk1) / n

            msetkmin, I1tk = np.min(phihattkmsevec), np.argmin(phihattkmsevec)
            phihattk = basisfunvals @ estcoeffstk[I1tk, :].T

            # ESTIMATOR INTEGRATED MEAN SQUARED ERRORS

            # SPECTRAL CUTOFF

            phihatsumvals = np.zeros((nxgrid, I1sc+1))
            phihatsumvals[:, 0] = np.tile(estcoeffs[0], nxgrid)
            for kk in range(1, I1sc+1):
                phihatsumvals[:, kk] = estcoeffs[kk] * np.sqrt(2) * np.cos((kk) * np.pi * xgrid)

            phihatscsum = phihatsumvals[:, 0:I1sc+1]
            phihatsc = np.sum(phihatscsum, axis=1)

            basisfunvals = np.zeros((nxgrid, J))
            basisfunvals[:, 0] = 1
            for jj in range(1, J):
                basisfunvals[:, jj] = np.sqrt(2) * np.cos((jj) * np.pi * xgrid)

            phihattk = basisfunvals @ estcoeffstk[I1tk, :].T

            phigrid = phi_x(J, xgrid, beta)

            values_spectral_cutoff.append(phihatsc)
            values_tikhonov.append(phihattk)

        SC_ISQB, SC_IVAR, SC_IMSE = integrated_mean_squared_error(values_spectral_cutoff, phigrid)
        TK_ISQB, TK_IVAR, TK_IMSE = integrated_mean_squared_error(values_tikhonov, phigrid)
        print([n, c, alpha, beta, SC_ISQB, SC_IVAR, SC_IMSE, TK_ISQB, TK_IVAR, TK_IMSE])

        new_row = {'n': n, 'delta': c, 'alpha': alpha, 'beta': beta, 'SC_ISQB': SC_ISQB, 'SC_IVAR': SC_IVAR, 'SC_IMSE': SC_IMSE, 'TK_ISQB': TK_ISQB, 'TK_IVAR': TK_IVAR, 'TK_IMSE': TK_IMSE}

        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index = True)
        results_df.head()

results_df

In [ ]:
results_df

,n,delta,alpha,beta,SC_ISQB,SC_IVAR,SC_IMSE,TK_ISQB,TK_IVAR,TK_IMSE
0,200,0,2,2,0.0234,0.0330,0.0564,0.0481,0.0205,0.0686
1,500,0,2,2,0.0171,0.0284,0.0455,0.0283,0.0227,0.0510
2,200,0.5,2,2,0.0193,0.0344,0.0537,0.0325,0.0243,0.0568
3,500,0.5,2,2,0.0202,0.0345,0.0547,0.0290,0.0241,0.0531
4,200,1,2,2,0.0216,0.0428,0.0644,0.0318,0.0257,0.0575
5,500,1,2,2,0.0218,0.0436,0.0654,0.0343,0.0265,0.0608
6,200,2,2,2,0.0261,0.0481,0.0742,0.0353,0.0271,0.0624
7,500,2,2,2,0.0304,0.0494,0.0798,0.0367,0.0279,0.0646
8,200,0,4,2,0.0298,0.0457,0.0755,0.0369,0.0266,0.0635
9,500,0,4,2,0.0268,0.0448,0.0716,0.0337,0.0267,0.0604
